<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-07 18:31:47--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9887 (9.7K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   9.66K  --.-KB/s    in 0s      

2022-06-07 18:31:47 (76.9 MB/s) - ‘utils.py’ saved [9887/9887]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.3 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
xyz_creator = ModelCreator(HUB_URL, 'XYZ-model')
xyz_model = xyz_creator.make_model(img_size=IMG_SIZE)

Model: "XYZ-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 600)               1229400   
                                                                 
 dropout (Dropout)           (None, 600)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                9616      
                                                                 
Total params: 24,739,368
Trainable params: 1,239,016
Non-trainable params: 23,500,352
_________________________________________________________________


In [10]:
callbacks = xyz_creator.get_callbacks()

# **First** day of training

In [ ]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=25,
                        class_weight=class_weight)

Epoch 1/25
519/519 [==============================] - 2665s 5s/step - loss: 1285.7012 - auc: 0.4276 - accuracy: 0.5987
Epoch 2/25
519/519 [==============================] - 172s 328ms/step - loss: 1400.2124 - auc: 0.4820 - accuracy: 0.6511
Epoch 3/25
519/519 [==============================] - 170s 324ms/step - loss: 1394.4506 - auc: 0.5243 - accuracy: 0.6875
Epoch 4/25
519/519 [==============================] - 174s 333ms/step - loss: 1170.0436 - auc: 0.5599 - accuracy: 0.7136
Epoch 5/25
519/519 [==============================] - 182s 349ms/step - loss: 975.6834 - auc: 0.5901 - accuracy: 0.7364
Epoch 6/25
519/519 [==============================] - 182s 347ms/step - loss: 831.1248 - auc: 0.6078 - accuracy: 0.7490
Epoch 7/25
519/519 [==============================] - 177s 338ms/step - loss: 727.5509 - auc: 0.6239 - accuracy: 0.7598
Epoch 8/25
519/519 [==============================] - 174s 333ms/step - loss: 558.0904 - auc: 0.6487 - accuracy: 0.7775
Epoch 9/25
519/519 [==================

In [ ]:
xyz_model.save('./drive/MyDrive/checkpoints/xyz-model-25.h5')

# ***Second*** day

In [11]:
xyz_model = tf.keras.models.load_model('drive/MyDrive/checkpoints/xyz-model-25.h5',
                                       custom_objects={'KerasLayer':hub.KerasLayer})

In [12]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=45,
                        initial_epoch=25,
                        class_weight=class_weight
                        )

Epoch 26/45
519/519 [==============================] - 2478s 5s/step - loss: 49.5260 - auc: 0.8471 - accuracy: 0.8924
Epoch 27/45
519/519 [==============================] - 171s 326ms/step - loss: 42.6935 - auc: 0.8563 - accuracy: 0.8988
Epoch 28/45
519/519 [==============================] - 172s 328ms/step - loss: 37.7512 - auc: 0.8592 - accuracy: 0.8990
Epoch 29/45
519/519 [==============================] - 176s 336ms/step - loss: 35.5559 - auc: 0.8685 - accuracy: 0.9054
Epoch 30/45
519/519 [==============================] - 166s 317ms/step - loss: 29.3628 - auc: 0.8784 - accuracy: 0.9115
Epoch 31/45
519/519 [==============================] - 174s 332ms/step - loss: 27.8082 - auc: 0.8871 - accuracy: 0.9155
Epoch 32/45
519/519 [==============================] - 172s 327ms/step - loss: 26.4545 - auc: 0.8853 - accuracy: 0.9141
Epoch 33/45
519/519 [==============================] - 166s 318ms/step - loss: 22.6312 - auc: 0.8943 - accuracy: 0.9159
Epoch 34/45
519/519 [=====================

In [13]:
xyz_model.save('./drive/MyDrive/checkpoints/xyz-model-45.h5')

In [14]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=55,
                        initial_epoch=45,
                        class_weight=class_weight)

Epoch 46/55
519/519 [==============================] - 170s 324ms/step - loss: 9.9243 - auc: 0.9416 - accuracy: 0.9374
Epoch 47/55
519/519 [==============================] - 173s 331ms/step - loss: 9.1274 - auc: 0.9439 - accuracy: 0.9367
Epoch 48/55
519/519 [==============================] - 184s 352ms/step - loss: 9.9053 - auc: 0.9384 - accuracy: 0.9366
Epoch 49/55
519/519 [==============================] - 167s 320ms/step - loss: 8.5145 - auc: 0.9476 - accuracy: 0.9425
Epoch 50/55
519/519 [==============================] - 171s 326ms/step - loss: 8.0599 - auc: 0.9474 - accuracy: 0.9417
Epoch 51/55
519/519 [==============================] - 170s 324ms/step - loss: 7.6836 - auc: 0.9489 - accuracy: 0.9434
Epoch 52/55
519/519 [==============================] - 169s 323ms/step - loss: 8.1299 - auc: 0.9510 - accuracy: 0.9425
Epoch 53/55
519/519 [==============================] - 169s 323ms/step - loss: 7.4536 - auc: 0.9528 - accuracy: 0.9438
Epoch 54/55
519/519 [===========================

In [15]:
xyz_model.save('./drive/MyDrive/checkpoints/xyz-model-55.h5')

In [16]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=60,
                        initial_epoch=55,
                        class_weight=class_weight)

Epoch 56/60
519/519 [==============================] - 169s 323ms/step - loss: 7.1294 - auc: 0.9554 - accuracy: 0.9469
Epoch 57/60
519/519 [==============================] - 174s 333ms/step - loss: 7.0125 - auc: 0.9538 - accuracy: 0.9455
Epoch 58/60
519/519 [==============================] - 170s 325ms/step - loss: 6.3978 - auc: 0.9559 - accuracy: 0.9486
Epoch 59/60
519/519 [==============================] - 175s 335ms/step - loss: 6.3172 - auc: 0.9572 - accuracy: 0.9480
Epoch 60/60
519/519 [==============================] - 171s 327ms/step - loss: 6.8798 - auc: 0.9558 - accuracy: 0.9451


In [17]:
xyz_model.save('./drive/MyDrive/checkpoints/xyz-model-60.h5')

In [18]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=65,
                        initial_epoch=60,
                        class_weight=class_weight)

Epoch 61/65
519/519 [==============================] - 187s 357ms/step - loss: 6.4984 - auc: 0.9576 - accuracy: 0.9461
Epoch 62/65
519/519 [==============================] - 180s 343ms/step - loss: 6.2245 - auc: 0.9579 - accuracy: 0.9495
Epoch 63/65
519/519 [==============================] - 176s 337ms/step - loss: 6.7042 - auc: 0.9563 - accuracy: 0.9476
Epoch 64/65
519/519 [==============================] - 172s 328ms/step - loss: 6.5438 - auc: 0.9578 - accuracy: 0.9479
Epoch 65/65
519/519 [==============================] - 172s 329ms/step - loss: 6.0676 - auc: 0.9604 - accuracy: 0.9489


In [19]:
xyz_model.save('./drive/MyDrive/checkpoints/xyz-model-65.h5')

In [ ]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=70,
                        initial_epoch=65,
                        class_weight=class_weight)

Epoch 66/70
519/519 [==============================] - 168s 321ms/step - loss: 6.0934 - auc: 0.9582 - accuracy: 0.9486
Epoch 67/70
391/519 [=====================>........] - ETA: 41s - loss: 6.1662 - auc: 0.9596 - accuracy: 0.9486

In [ ]:
xyz_model.save('./drive/MyDrive/checkpoints/xyz-model-70.h5')